In [6]:
from flask import Flask, request
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from requests_html import HTMLSession, AsyncHTMLSession 

In [2]:
keyword = '갤럭시'
url_sh1 = f'https://m.bunjang.co.kr/search/products?q={keyword}'  # 번개장터
url_sh2 = f'https://m.joongna.com/search-list/product?searchword={keyword}'  # 중고나라
url_new1 = f'https://search.shopping.naver.com/search/all?query={keyword}'  # 네이버쇼핑
url_new2 = f'https://www.coupang.com/np/search?component=&q={keyword}'  # 쿠팡
TOTAL_NUM = 10
NUM_FROM_A_SITE = 20

CLS_NAME = ['title', 'price', 'image', 'link']
CLS_SH1 = [['div', 'sc-fcdeBU iVCsji'], ['div', 'sc-gmeYpB iBMbn'], '상품 이미지']
CLS_SH2 = [['span', 'ProductItemV2_title__1KDby'], ['p', 'ProductItemV2_price__1a5yb mt3'], '이미지']
CLS_NEW1 = [['div', 'basicList_title__3P9Q7'], ['span', 'price_num__2WUXn']]
CLS_NEW2 = [['div', 'name'], ['strong', 'price-value'], ['img', 'search-product-wrap-img']]

In [3]:

def get_shs_from_bs(bs, cls_names, url):    
#     session = AsyncHTMLSession()
#     resp = await session.get(url_sh1)
#     await resp.html.arender(sleep = 1) 
#     bs = BeautifulSoup(resp.html.raw_html, 'html.parser')

    df = pd.DataFrame(columns=CLS_NAME)
    for i, cls in enumerate(cls_names[:2]):
        df[CLS_NAME[i]] = list(map(lambda x: x.get_text(), bs.find_all(*cls)[:NUM_FROM_A_SITE]))
    df[CLS_NAME[2]] = list(map(lambda x: x['src'], bs.find_all('img', alt=cls_names[2])[:NUM_FROM_A_SITE]))
    df[CLS_NAME[3]] = url
    return df


def get_new_from_naver(bs, cls_names, url):    
#     session = AsyncHTMLSession()
#     resp = await session.get(url_sh1)
#     await resp.html.arender(sleep = 1) 
#     bs = BeautifulSoup(resp.html.raw_html, 'html.parser')

    df = pd.DataFrame(columns=CLS_NAME)
    for i, cls in enumerate(cls_names[:2]):
        df[CLS_NAME[i]] = list(map(lambda x: x.get_text(), bs.find_all(*cls)[:NUM_FROM_A_SITE]))
        
    #############TODO
    imgs = list(map(lambda x: x['src'], bs.find_all('a',alt=df[CLS_NAME[0]])))
    if len(imgs) > NUM_FROM_A_SITE:
        df[CLS_NAME[2]] = imgs[:NUM_FROM_A_SITE] ## 이렇게 나와야함
    df[CLS_NAME[3]] = url
    return df


def get_new_from_coupang(bs, cls_names, url):    
    df = pd.DataFrame(columns=CLS_NAME)
    for i, cls in enumerate(cls_names[:2]):
        df[CLS_NAME[i]] = list(map(lambda x: x.get_text(), bs.find_all(*cls)[:NUM_FROM_A_SITE]))
        
    df[CLS_NAME[2]] = list(map(lambda x: x['src'], bs.find_all(*cls_names[2])[:NUM_FROM_A_SITE]))
    df[CLS_NAME[3]] = url
    return df


def string_price_to_num(str_price):
    str_price = str_price.strip().strip("원").split(",")
    s = ''
    for string in str_price:
        s += string
    return int(s)


def alysis_dfs(df1, df2):
    df = pd.concat([df1, df2], axis=0)

    df['price'] = list(map(string_price_to_num, df['price']))  # 가격 숫자로
    df = df.sort_values('price')[:TOTAL_NUM].reset_index()  # 가격 정렬
    items = []
    for sh in list(df.iterrows()):
        items.append({
            'title': sh[1]['title'],
            'price': sh[1]['price'],
            'image': sh[1]['image'],
            'link': sh[1]['link']
        })
    return items

        

In [4]:
## 번개장터
session = AsyncHTMLSession()
resp = await session.get(url_sh1)
await resp.html.arender(sleep = 1) 
bs = BeautifulSoup(resp.html.raw_html, 'html.parser')

sh_df = get_shs_from_bs(bs, CLS_SH1, url_sh1)

## 중고나라
session = AsyncHTMLSession()
resp = await session.get(url_sh2)
await resp.html.arender(sleep = 1) 
bs = BeautifulSoup(resp.html.raw_html, 'html.parser')

sh_df_ = get_shs_from_bs(bs, CLS_SH2, url_sh2)
sh_items = alysis_dfs(sh_df, sh_df_)
print(sh_items)
    
## 네이버쇼핑
session = AsyncHTMLSession()
resp = await session.get(url_new1)
await resp.html.arender(sleep = 1) 
bs = BeautifulSoup(resp.html.raw_html, 'html.parser')
new_df = get_new_from_naver(bs, CLS_NEW1, url_new1)
print(new_df)

## 하나쯤 더 있으면 좋은데 다나와는 안됨
page = requests.get(url_new2)
bs = BeautifulSoup(page.text, 'html.parser')
new_df_ = get_new_from_coupang(bs, CLS_NEW2, url_new2)
new_items = alysis_dfs(new_df, new_df_)
print(new_items)

[{'title': '갤럭시s10 128GB 블랙', 'price': 100000, 'image': 'https://media.bunjang.co.kr/product/185989124_1_1650775162_w292.jpg', 'link': 'https://m.bunjang.co.kr/search/products?q=갤럭시'}, {'title': '갤럭시 버즈프로 단순개봉 풀박스 새제품 안산버즈프로', 'price': 105000, 'image': 'https://img2.joongna.com/media/original/2022/03/06/1646554543613lIG_NFPMF.jpg?impolicy=thumb&size=150', 'link': 'https://m.joongna.com/search-list/product?searchword=갤럭시'}, {'title': '갤럭시엑티브2 40mm (R830) 442174', 'price': 110000, 'image': 'https://media.bunjang.co.kr/product/186838292_1_1651568248_w292.jpg', 'link': 'https://m.bunjang.co.kr/search/products?q=갤럭시'}, {'title': '갤럭시S8 64g S급 블랙 최저가 선착순 안산s8', 'price': 119000, 'image': 'https://img2.joongna.com/media/original/2022/04/26/1650956822489oBw_TbxFH.jpg?impolicy=thumb&size=150', 'link': 'https://m.joongna.com/search-list/product?searchword=갤럭시'}, {'title': '삼성 갤럭시a40 화이트 무잔상 A급 A405 [순천중고폰]', 'price': 120000, 'image': 'https://media.bunjang.co.kr/product/186838259_1_1651568230_w29

In [7]:
json.dumps({
        "new":new_items,
        "sh":sh_items
        })

'{"new": [{"title": "\\uc6b0\\uba3c\\uc2a49999 \\uac24\\ub7ed\\uc2dc\\uc6cc\\uce58 4 \\ubca0\\uc824\\ub9c1 \\ucf00\\uc774\\uc2a4 \\uac24\\uc6cc\\uce584 \\ubca0\\uc824\\ub9c1 \\ucee4\\ubc84 \\ud504\\ub808\\uc784 40 44 mm \\ucf00\\uc774\\uc2a4", "price": 9900, "image": "//thumbnail9.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/1651/07f8dcedff226e34e520ca1b1921d53711a135689fff72f5ed05ad69c14b.jpg", "link": "https://www.coupang.com/np/search?component=&q=\\uac24\\ub7ed\\uc2dc"}, {"title": "\\ube44\\uc698\\ub4dc\\ub9c8\\ucf13 \\uac24\\ub7ed\\uc2dcX\\ucee4\\ubc845 \\ucf00\\uc774\\uc2a4 \\ubaa9\\uac78\\uc774 \\ud31d\\uc787 \\uc9c0\\uac11 XCOVER5 \\uc7bc\\uafb8\\ub7ec\\uae30\\ud3f0 \\uc2e0\\ube44\\ud0a4\\uc988\\ud3f02", "price": 12800, "image": "//img1a.coupangcdn.com/image/coupang/search/blank1x1.gif", "link": "https://www.coupang.com/np/search?component=&q=\\uac24\\ub7ed\\uc2dc"}, {"title": "\\uc2e0\\uc9c0\\ubaa8\\ub8e8 \\uac24\\ub7ed\\uc2dc\\uc6cc\\uce58 9H \\uace0\\uac